# Connecting to DynamoDB
We'll use this to explore connecting to our remote DynamoDB

In [ ]:
import pydantic
import uuid

from datetime import datetime
from typing import Union, List
from pydantic.types import StrictBool

# === Data Models ===


class StatusResponse(pydantic.BaseModel):
    status: str
    uptime_seconds: float
    version: str

class chatClientMessage(pydantic.BaseModel):
    user_id: int
    conversation_id: int
    message: str
    client: str
    timestamp: datetime = datetime.now()


class conversationMessage(pydantic.BaseModel):
    message_id: str = uuid.uuid4()
    user_id: str
    conversation_id: Union[str, None] = None
    message: str
    timestamp: datetime = datetime.now()
    retrieved_documents: List = None
    llm_prompt: Union[str, None] = None

class user(pydantic.BaseModel):
    user_id: str = uuid.uuid4()
    email: str
    created_at: datetime = datetime.now()
    admin : StrictBool = False



In [ ]:
test_message = conversationMessage(
    user_id='1',
    message='Hi I would like some help with my angry dog',
    timestamp=datetime.utcnow(),
    retrieved_documents=[],
)

test_message

In [ ]:
test_user = user(
    email='john@bob.com',
)

test_user

In [ ]:
os.getenv('DB_SECRET_ACCESS_KEY')

In [ ]:
import boto3
import json
import os

# Initialize the DynamoDB client
dynamodb = boto3.resource('dynamodb', region_name='eu-west-2',aws_access_key_id=os.getenv('DB_ACCESS_KEY_ID'), aws_secret_access_key=os.getenv('DB_SECRET_ACCESS_KEY'))
message_table = dynamodb.Table('caddyMessages')
users_table = dynamodb.Table('caddyUsers')


def store_message(message : conversationMessage, table):

    # Storing in DynamoDB
    response = table.put_item(
        Item={
            'conversationId': str(message.conversation_id),
            'messageId': str(message.message_id),
            'text': message.message,
            'timestamp': message.timestamp.isoformat(),
        }
    )

    return {
        'statusCode': 200,
        'body': json.dumps({'message': 'Message stored successfully!'})
    }


In [ ]:
def store_user(user : conversationMessage, table):

    # Storing in DynamoDB
    response = table.put_item(
        Item={
            'userId': str(user.user_id),
            'email': user.email,
            'createdat': user.created_at.isoformat(),
            'is_admin': user.is_admin
        }
    )

    return {
        'statusCode': 200,
        'body': json.dumps({'message': 'User stored successfully!'})
    }

In [ ]:
store_message(test_message, message_table)

In [ ]:
store_user(test_user, users_table)

In [ ]:
find_item = message_table.get_item(Key={'messageId':1,
                                'conversationId':1})
find_item